In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [0]:
df=spark.read.load("/FileStore/tables/part_00000_4fb2c33a_49da_4456_991f_f96fb8ec06d9_c000.csv",format="csv",sep=",",header="true",inferSchema="true")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [0]:
df.printSchema()

root
 |-- ORDER_ID: integer (nullable = true)
 |-- SUBSCRIBER_ID: integer (nullable = true)
 |-- STATUS_DATE: integer (nullable = true)
 |-- STATUS_TIME: integer (nullable = true)
 |-- STATUS: string (nullable = true)



In [0]:
df.createOrReplaceTempView("orders")

In [0]:
result_df = spark.sql("""
    SELECT 
        ORDER_ID AS ORDER_ID,
        SUBSCRIBER_ID AS SUBSCRIBER_ID,
        first(STATUS) AS STATUS,
        max(
            CASE
            WHEN STATUS = 'CREATED' OR STATUS = 'POOL' THEN 
            TO_TIMESTAMP(CONCAT(STATUS_DATE,lpad(STATUS_TIME, 6, '0')),'yyyyMMddHHmmss')
            else NULL
            END 
        ) as START_DATE,
         max(
            CASE
            WHEN STATUS = 'COMPLETED' or STATUS="CANCELLED" THEN 
            TO_TIMESTAMP(CONCAT(STATUS_DATE,lpad(STATUS_TIME, 6, '0')),'yyyyMMddHHmmss')
            else NULL
            END 
        ) as END_DATE,
        
            CASE when END_DATE IS NOT NULL THEN  round((unix_timestamp(End_Date) - unix_timestamp(Start_Date)) / 3600 ,6)
            ELSE 0.0
            END  AS DURATION
       
        from orders
        GROUP BY ORDER_ID,SUBSCRIBER_ID
        ORDER BY 
        ORDER_ID ASC, min(to_timestamp(concat(STATUS_DATE, lpad(STATUS_TIME, 6, '0')), 'yyyyMMddHHmmss')) DESC
        limit 10
  
""")

result_df.show()







+--------+-------------+---------+-------------------+-------------------+---------+
|ORDER_ID|SUBSCRIBER_ID|   STATUS|         START_DATE|           END_DATE| DURATION|
+--------+-------------+---------+-------------------+-------------------+---------+
|  100001|       200574| ASSIGNED|2023-02-22 09:53:06|               null|      0.0|
|  100002|       200121|     POOL|2023-02-22 09:37:12|               null|      0.0|
|  100003|       200432| ASSIGNED|2023-02-22 09:40:01|               null|      0.0|
|  100004|       200234|COMPLETED|2023-02-22 09:37:58|2023-02-22 10:44:26| 1.107778|
|  100005|       200546|     POOL|2023-02-22 09:47:27|               null|      0.0|
|  100006|       200369| ASSIGNED|2023-02-22 10:54:13|               null|      0.0|
|  100007|       200486| ASSIGNED|2023-02-22 10:54:48|               null|      0.0|
|  100008|       200190| ASSIGNED|2023-02-22 09:43:31|               null|      0.0|
|  100009|       200058|COMPLETED|2023-02-22 09:49:51|2023-02-22 